# Preprocessing

In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [ ]:
# Be able to view more of dataframes

pd.options.display.max_rows = 500
pd.options.display.max_columns = 50

## Functions

In [ ]:
def load_dataset():
    """ Reads dataset csv and returns pandas dataframe """
    
    filepath = "../Resources/charity_data.csv"

    df = pd.read_csv(filepath, encoding="utf-8", low_memory=False)
    
    return df

In [ ]:
def clean_dataset(a_df):
    """ Returns deduped, na-dropped, useless column dropped, index-reset dataframe """    
    
    a_df = a_df.drop_duplicates()   
        
    a_df = a_df.dropna()
    
    a_df = a_df.drop(columns=["EIN", "NAME"])
        
    a_df = a_df.reset_index(drop=True)
    
    return a_df   

In [ ]:
def examine_dataset(a_df):
    """ Provides summary info and visualizations of dataset """
    
    a_df.info()
    
    print("\n\n")
                  
    # Determine the number of unique values in each column.

    for col in a_df.columns:
        if (a_df[col].nunique() > 10):
            print(f"{col}\n\n{a_df[col].value_counts()}\n\n")        

In [ ]:
def reduce_cats(a_col, a_cutoff):
    """ Inputs are a series and a cutoff value for 'Other' """
    
    print(f"BEFORE: \n\n{df[a_col].value_counts()}\n\n")

    types_to_replace = (df[a_col].value_counts().loc[lambda x: x < int(a_cutoff)]).keys().tolist()

    for code in types_to_replace:        
        df[a_col] = df[a_col].replace(code, "Other")

    # Check to make sure binning was successful

    print(f"AFTER: \n\n{df[a_col].value_counts()}\n\n")    

In [ ]:
def encode_df(a_df):
    """ Returns one-hot encoded dataframe """
    
    categorical_list = a_df.dtypes[a_df.dtypes == "object"].index.tolist() 
    
    print(f"CATEGORIES FOR EACH CATEGORICAL FEATURE ENCODED:\n\n{a_df[categorical_list].nunique()}\n\n")
    
    concat_list = []
    
    for categorical in categorical_list:
        
        concat_list.append(pd.get_dummies(a_df[categorical], prefix=categorical, prefix_sep='_'))        
    
    concat_list.append(a_df["IS_SUCCESSFUL"])

    return pd.concat(concat_list, axis=1)  

In [ ]:
def pre_process(a_df):
    """ Make X,y ... train_test_split ... scale, fit and transform """
    
    # Split our preprocessed data into our features and target arrays

    y = enc_df["IS_SUCCESSFUL"]
    X = enc_df.drop(["IS_SUCCESSFUL"], axis=1)

    # Split the preprocessed data into a training and testing dataset

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 
    
    # Create a StandardScaler instances
    
    scaler = StandardScaler()

    # Fit the StandardScaler

    X_scaler = scaler.fit(X_train)

    # Scale the data

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    print(f"TRAIN SCALED SHAPE: {X_train_scaled.shape}")
    print(f"TEST SCALED SHAPE: {X_test_scaled.shape}")

    input_dim = X_train_scaled.shape[1]  
    
    return input_dim, X_train_scaled, X_test_scaled, y_train, y_test

## Call functions

In [ ]:
# Make dataframe and examine

df = clean_dataset(load_dataset())

examine_dataset(df)

In [ ]:
# Bin columns with > 10 unique values

reduce_cats("APPLICATION_TYPE", 500)

reduce_cats("ASK_AMT", 25_000)

reduce_cats("CLASSIFICATION", 1800)

In [ ]:
df.head()

In [ ]:
enc_df = encode_df(df)

enc_df

In [ ]:
# Preprocess dataframe

input_dim, X_train_scaled, X_test_scaled, y_train, y_test = pre_process(enc_df)

# Compile, Train and Evaluate the Model

## Functions

In [ ]:
def callback_log():
    """ Saves weights every 5 epochs """
    
    tf.keras.callbacks.ModelCheckpoint("../Models/weights.h5", save_weights_only=True, save_freq=5)    

In [ ]:
def make_nn(input_dim=43, num_layers=2, num_units=100, num_epochs=10):
    """ Makes sequential nn, compiles, fits, saves, and reports on loss and accuracy """    
    
    nn = tf.keras.models.Sequential()

    # First layer 
    
    nn.add(tf.keras.layers.Dense(units=num_units, input_dim=input_dim, activation="relu"))
    
    # Hidden layers
    
    for layer in range(1, num_layers):
        
        nn.add(tf.keras.layers.Dense(units=num_units, activation="relu"))

    # Output layer

    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    nn.summary()
    
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 
    
    fit_model = nn.fit(X_train_scaled, y_train, epochs=num_epochs, callbacks=callback_log())   
    
    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
    
    print(f"\n\nLoss: {model_loss}, Accuracy: {model_accuracy}")    

    nn.save("../Models/AlphabetSoupCharity.h5")

## Call functions

In [ ]:
make_nn()

# Best result: 

### Loss: 0.5545602440834045, Accuracy: 0.7287463545799255